Import the required packages
and define the global variables

In [ ]:
#!pip install BeautifulSoup
#!pip install json

In [ ]:
import os
from PIL import Image
from io import BytesIO
import requests
import json
from bs4 import BeautifulSoup
import re
from copy import deepcopy

articles_count = 200

base_url = "http://www.wikihow.com"
api_url = 'https://www.wikihow.com/api.php?'
use_colab = False


In [ ]:
if (use_colab):
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_directory = '/content/drive/MyDrive/Colab/datasets/wikihow'
    workspace = '/content/drive/MyDrive/Colab/wikihow'
else:
     dataset_directory = '/kuacc/users/asafa22/google-drive/wikihow'
     workspace = '/kuacc/users/asafa22/workspace/wikihow/Siamese'

article_dictionary_file_path = os.path.join(dataset_directory, 'articles.json')

Read article dictionary file if exists

In [ ]:
try:
    old_article_dictionaries_file = open(article_dictionary_file_path,)
    old_article_dictionaries = json.load(old_article_dictionaries_file)
    print('Article dictionary file loaded successfully. Number of found entries ', len(old_article_dictionaries))
except:
    print('Couldn\'t read old article dictionary file')
    old_article_dictionaries = dict()

article_dictionaries = dict()

Implemet function to retrieve random artilces from WikiHow

In [ ]:
def retrieve_random_articles(article_count):
    articles = set()
    max = 500
    limit = max

    if article_count < max:
        limit = article_count
    else:
        limit = max

    query = 'action=query&generator=random&grnnamespace=0&prop=info&format=json&grnlimit=' + \
        str(limit)

    while len(articles) < article_count:
        response = requests.get(api_url+query).json()
        pages = response['query']['pages']
        for page in pages:
            id = pages[page]['pageid']
            title = pages[page]['title']
            articles.add(title)

    return articles


Implement function to retrieve wikiHow article content

In [ ]:
def retrieve_article(title):
    article_dictionary = dict()
    step_dictionaries = []

    # retrieve categories #
    properties_query = 'action=query&prop=categories&format=json&titles='+title
    content_query = 'action=parse&prop=text&format=json&page='+title

    response = requests.get(api_url+properties_query).json()
    # retrieve and parse categories #
    categories = []
    pageid = next(iter(response['query']['pages']))
    category_jsons = response['query']['pages'][pageid]['categories']
    for category_json in category_jsons:
        category = category_json['title']
        categories.append(category)

    if 'Category:Stub' in categories:
        print("Stub article")
        return article_dictionary

    # retrieve and parse html #
    response = requests.get(api_url+content_query).json()
    html = response['parse']['text']['*']
    soup = BeautifulSoup(html, features="html.parser")

    step_index = 0
    for step in soup.find_all("li"):
        step_dictionary = dict()
        image_src = ""
        step_text = step.text
        step_img_divs = step.find_all('div')
        for step_img_div in step_img_divs:
            div_class = step_img_div['class']
            if 'largeimage' in div_class:
                # step image #
                images = step_img_div.find_all('img')
                for image in images:
                    image_src = image['src']
                    if not image_src.startswith('http'):
                        image_src = base_url+image_src

            step_dictionary['step_id'] = step_index
            step_dictionary['step'] = step_text
            step_dictionary['image'] = image_src
            step_dictionaries.append(step_dictionary)
            step_index += 1

    if len(step_dictionaries) == 0:
        return article_dictionary
    
    article_dictionary['title'] = title
    article_dictionary['categories'] = categories
    article_dictionary['steps'] = step_dictionaries

    return article_dictionary


In [ ]:
def download_article_images(article):
    title = article['title']
    
    print('title ', title)
    article_dir_name = re.sub(r'[^A-Za-z0-9]+', '_', title)
    article_relative_dir = os.path.join('images',article_dir_name)
    article_dir = os.path.join(
        dataset_directory, article_relative_dir)
    
    if not os.path.exists(article_dir):
        os.makedirs(article_dir)

    steps = article['steps'].copy()
    for step in steps:
            step_id = step['step_id']
            image_url = step['image']
            image_paths = os.path.split(image_url)
            image_name = image_paths[-1]
            image_name = image_name.replace('Image:', '')
            #image_extension = image_name.split('.')[-1]
            image_extension = 'jpeg'
            image_name = str(step_id)+'.'+image_extension # to avoid long names
            image_file_relative_path = os.path.join(article_relative_dir, image_name)
            image_file_path = os.path.join(article_dir, image_name)
            image = requests.get(image_url).content
            try:
              image_object = Image.open(BytesIO(image))
              rgb_image = image_object.convert('RGB')
              rgb_image.save(image_file_path)
              step['image'] = image_file_relative_path
            except:
              print('Couldnt convert image')
              steps.remove(step)
            #with open(image_file_path, 'wb') as image_file:
            #    image_file.write(image)
            #    image_file.close()
            
    article['steps'] = steps
    return article


Retrieve n random article titles

In [ ]:
print(articles_count)
articles = retrieve_random_articles(articles_count)

Retrieve the articles content

In [ ]:
articles_to_delete = []
for title in articles:
    if title in old_article_dictionaries:
        print('Title ', title, ' already processed')
        continue
    
    print('Start processing article ', title)
    try:
        article_dictionary = retrieve_article(title)
        if article_dictionary:
            article_dictionaries[title] = article_dictionary
    except:
        print('Couldn\'t parse article')
    print('Finished processing article ', title)

print('Number of articles = ', len(articles))



Download the article images

In [ ]:
articles_to_delete = []
for title in article_dictionaries:
    print('Start downloading images for ', title)
    try:
        
        article = download_article_images(article_dictionaries[title])
        if len(article['steps'])!=0:
          article_dictionaries[title] = article
        else:
          articles_to_delete.append(title)
    except:
        print('Couldn\'t download images')
        articles_to_delete.append(title)
    print('Finished downloading images for ', title)

print('Number of articles with downloaded images= ', len(article_dictionaries))

# delete failed articles #
for title in articles_to_delete:
    del article_dictionaries[title]

print article dictionaries to file

In [ ]:
article_dictionaries.update(old_article_dictionaries)
step_count = 0
for key in article_dictionaries:
  steps = article_dictionaries[key]['steps']
  step_count+=len(steps)
print('Total number of processed articles (including old ones): ',len(article_dictionaries),' and number of steps: ',step_count)
article_file = open(article_dictionary_file_path, "w")

article_file.write(json.dumps(article_dictionaries))
article_file.close()


In [ ]:
import os
def check_images():
  image_count = 0
  failed_image_count=0
  from PIL import Image
  folder_path = dataset_directory+"/images"
  extensions = []
  for fldr in os.listdir(folder_path):
      sub_folder_path = os.path.join(folder_path, fldr)
      for filee in os.listdir(sub_folder_path):
          file_path = os.path.join(sub_folder_path, filee)
          print('** Path: {}  **'.format(file_path), end="\r", flush=True)
          image_count+=1
          try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
          except:
            print('failed to open image: ', file_path)
            failed_image_count+=1
          if filee.split('.')[1] not in extensions:
              extensions.append(filee.split('.')[1])
  print("Number of scanned images = ", image_count)
  print("Number of failed images = ", failed_image_count)
#check_images()

In [25]:
import os
import copy
def filter_images():
    articles_to_delete = []
    article_file = open(article_dictionary_file_path,)
    article_dictionary = json.load(article_file)
    article_file.close()
    
    print("number of articles in dictonary = ", len(article_dictionary))
    for article in article_dictionary.keys():
        steps = copy.deepcopy(article_dictionary[article]['steps'])
        filtered_steps = []
        for step in steps:
            image = step['image']
            if not 'http' in image:
                filtered_steps.append(step)
        article_dictionary[article]['steps'] = copy.deepcopy(filtered_steps)
        if len(steps) ==0:
            print('Article has no steps', article)
            articles_to_delete.append(article)
        
    for article in articles_to_delete:
         article_dictionary.pop(article)
    
    print("number of articles in dictonary after filtering = ", len(article_dictionary))
    article_file = open(article_dictionary_file_path, "w")
    article_file.write(json.dumps(article_dictionary))
    article_file.close()

filter_images()

number of articles in dictonary =  2368
number of articles in dictonary after filtering =  2368


In [ ]:
import os
from PIL import Image
def convert_images():
  image_count = 0
  failed_image_count = 0
  folder_path = dataset_directory+"/images"
  extensions = []
  for fldr in os.listdir(folder_path):
  #for fldr in (['Manage_Mental_Illness_in_a_Marriage']):
      sub_folder_path = os.path.join(folder_path, fldr)
      for filee in os.listdir(sub_folder_path):
          file_path = os.path.join(sub_folder_path, filee)
          print('** Path: {}  **'.format(file_path), end="\r", flush=True)
          image_count+=1
          try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
            rgb_im.save(file_path)
          except:
            print('failed to open image: ', file_path)
            failed_image_count+=1
          if filee.split('.')[1] not in extensions:
              extensions.append(filee.split('.')[1])
  print("Number of scanned images = ", image_count)
  print("Number of failed images = ", failed_image_count)
#convert_images()